In [29]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import lightgbm as lgb
#import xgboost as xgb
import pickle
import sys
sys.path.append('..')

In [30]:
notebook_name = os.path.basename(os.getcwd())
timestamp="07"

In [31]:
path_movie_dir = '../data/movie/fps30_mask/'
path_save_dir = os.path.join('../data/05_Train_regression/', timestamp)
if not os.path.exists(path_save_dir):
    os.makedirs(path_save_dir)
path_train_file = os.path.join(path_save_dir, 'train.csv')
path_model_file = os.path.join(path_save_dir, 'model.pkl')
path_test_file = os.path.join(path_save_dir,'test.csv')

In [32]:
path_label_name_file = '../data/labels/labels_num.csv'
path_action_dir = os.path.join('../data/01_DataProcessing/', timestamp)
path_point_dir = os.path.join('../data/02_GetPoints/', timestamp)
path_split_dir = os.path.join('../data/04_Split_new/', timestamp)

path_train_file_dir = os.path.join(path_split_dir, 'train')
path_test_file_dir = os.path.join(path_split_dir, 'test')
# path_model_dir = '../data/05_train/'

In [33]:
file_names = ['fall','sitdown', 'standup']
# file_names = ['forehand']
movie_names = [file_name + '.MOV' for file_name in file_names]
points_file_names = [file_name + '.csv' for file_name in file_names]

In [34]:
objective = 'action_gt_num'

In [35]:
features = ['RShoulder','RHip','RKnee','Neck','Nose',
           'LShoulde','LHip','LKnee']

In [36]:
target_columns = []
for feature in features:
    target_columns.append(feature + '_x')
    target_columns.append(feature + '_y')
target_columns

['RShoulder_x',
 'RShoulder_y',
 'RHip_x',
 'RHip_y',
 'RKnee_x',
 'RKnee_y',
 'Neck_x',
 'Neck_y',
 'Nose_x',
 'Nose_y',
 'LShoulde_x',
 'LShoulde_y',
 'LHip_x',
 'LHip_y',
 'LKnee_x',
 'LKnee_y']

In [37]:

num = 0
path_file = os.path.join(path_train_file_dir, points_file_names[num])
data_df = pd.read_csv(path_file, index_col='frame_num')
# data_df = pd.read_csv(path_file, index_col='frame_num')

In [38]:
frame_length = 15
X = data_df.drop(columns=[objective])
X

,Nose_x,Nose_y,Neck_x,Neck_y,RShoulder_x,RShoulder_y,RElbow_x,RElbow_y,RWrist_x,RWrist_y,...,LAnkle_x,LAnkle_y,REye_x,REye_y,LEye_x,LEye_y,REar_x,REar_y,LEar_x,LEar_y
frame_num,,,,,,,,,,,,,,,,,,,,,
0,0.168722,0.361548,0.184287,0.426038,0.140705,0.432487,0.125140,0.516324,0.112688,0.574365,...,0.237209,0.787183,0.162496,0.348650,0.178061,0.348650,0.150044,0.361548,0.196740,0.361548
1,0.168722,0.361548,0.184287,0.426038,0.143818,0.432487,0.128253,0.516324,0.115801,0.574365,...,0.240322,0.793632,0.162496,0.355099,0.178061,0.348650,0.153157,0.361548,0.196740,0.361548
2,0.171835,0.361548,0.184287,0.426038,0.143818,0.438936,0.128253,0.522773,0.118914,0.580814,...,0.249661,0.793632,0.165609,0.355099,0.181174,0.348650,0.153157,0.361548,0.199853,0.361548
3,0.174948,0.361548,0.187401,0.419589,0.143818,0.432487,0.131366,0.522773,0.122027,0.580814,...,0.252774,0.793632,0.168722,0.355099,0.184287,0.355099,0.156270,0.361548,0.199853,0.361548
4,0.174948,0.361548,0.187401,0.419589,0.146931,0.432487,0.134479,0.516324,0.125140,0.580814,...,0.255887,0.787183,0.168722,0.355099,0.184287,0.355099,0.159383,0.367997,0.202966,0.361548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,0.103348,0.845224,0.072218,0.864571,0.056653,0.883918,0.062879,0.903265,0.059766,0.896816,...,0.196740,0.929061,0.084670,0.851673,0.100235,0.825877,0.072218,0.845224,0.106461,0.806530
1507,0.103348,0.845224,0.072218,0.864571,0.056653,0.883918,0.062879,0.903265,0.059766,0.896816,...,0.196740,0.929061,0.084670,0.851673,0.100235,0.825877,0.072218,0.845224,0.106461,0.806530
1508,0.103348,0.845224,0.072218,0.864571,0.056653,0.883918,0.062879,0.903265,0.059766,0.896816,...,0.196740,0.929061,0.084670,0.851673,0.100235,0.825877,0.072218,0.845224,0.106461,0.806530


In [39]:
points_file_names[0]

'fall.csv'

In [40]:
points_file_name = points_file_names[0]
path_file = os.path.join(path_train_file_dir, points_file_name)
data_df = pd.read_csv(path_file, index_col='frame_num')

In [41]:
units_df = None
new_data_df = None
for num, points_file_name in enumerate(points_file_names):
    print(points_file_name)
    path_file = os.path.join(path_train_file_dir, points_file_name)
    data_df = pd.read_csv(path_file, index_col='frame_num')
    units_df = None
    for idx in data_df.index[frame_length:len(data_df)]:
        unit_df = data_df[idx - frame_length: idx]
        y = unit_df[objective]
        Y=y.loc[idx-1]
        X = unit_df.drop(columns=[objective]).loc[:, target_columns]
        flatten_data=X.to_numpy().reshape(1,-1)
        A=['RShoulder_x','RShoulder_y', 'RHip_x', 'RHip_y','RKnee_x',
 'RKnee_y', 'Neck_x', 'Neck_y', 'Nose_x', 'Nose_y', 'LShoulde_x', 'LShoulde_y',
 'LHip_x', 'LHip_y', 'LKnee_x', 'LKnee_y']
        B=[a+"_"+str(i-14) for i in range(15) for a in A]
        X=pd.DataFrame(flatten_data,columns=B)
        X["action_gt_num"]=Y
        unit_df = X
        if units_df is None:
            units_df = unit_df
        else:
            units_df = pd.concat([units_df, unit_df], axis=0, sort=False)
    path_save_file = os.path.join(path_save_dir, points_file_name)
    units_df.to_csv(path_save_file)
    if new_data_df is None:
        new_data_df = units_df
    else:
        new_data_df = pd.concat([new_data_df, units_df], axis=0, sort=False)
train_data_df = new_data_df.copy()
train_data_df.to_csv(path_train_file)

fall.csv
sitdown.csv
standup.csv


In [42]:
train_data_df = new_data_df.copy()

In [43]:
train_data_df.groupby('action_gt_num').count()

,RShoulder_x_-14,RShoulder_y_-14,RHip_x_-14,RHip_y_-14,RKnee_x_-14,RKnee_y_-14,Neck_x_-14,Neck_y_-14,Nose_x_-14,Nose_y_-14,...,Neck_x_0,Neck_y_0,Nose_x_0,Nose_y_0,LShoulde_x_0,LShoulde_y_0,LHip_x_0,LHip_y_0,LKnee_x_0,LKnee_y_0
action_gt_num,,,,,,,,,,,,,,,,,,,,,
0,1021,1021,1021,1021,1021,1021,1021,1021,1021,1021,...,1021,1021,1021,1021,1021,1021,1021,1021,1021,1021
1,948,948,948,948,948,948,948,948,948,948,...,948,948,948,948,948,948,948,948,948,948
2,185,185,185,185,185,185,185,185,185,185,...,185,185,185,185,185,185,185,185,185,185
3,282,282,282,282,282,282,282,282,282,282,...,282,282,282,282,282,282,282,282,282,282
4,304,304,304,304,304,304,304,304,304,304,...,304,304,304,304,304,304,304,304,304,304


In [44]:
X_train_data_df = train_data_df.drop([objective], axis=1)
y_train_data_df = train_data_df[objective]

In [45]:

train_data_df['dummy'] = 1
train_data_df.groupby(objective).count()['dummy']

action_gt_num
0    1021
1     948
2     185
3     282
4     304
Name: dummy, dtype: int64

In [46]:
train_data_df.head()

,RShoulder_x_-14,RShoulder_y_-14,RHip_x_-14,RHip_y_-14,RKnee_x_-14,RKnee_y_-14,Neck_x_-14,Neck_y_-14,Nose_x_-14,Nose_y_-14,...,Nose_x_0,Nose_y_0,LShoulde_x_0,LShoulde_y_0,LHip_x_0,LHip_y_0,LKnee_x_0,LKnee_y_0,action_gt_num,dummy
0,0.140705,0.432487,0.174948,0.587263,0.181174,0.683998,0.184287,0.426038,0.168722,0.361548,...,0.196740,0.374446,0.255887,0.426038,0.243435,0.567916,0.259001,0.683998,0,1
0,0.143818,0.432487,0.174948,0.587263,0.181174,0.683998,0.184287,0.426038,0.168722,0.361548,...,0.199853,0.374446,0.259001,0.426038,0.243435,0.567916,0.259001,0.683998,0,1
0,0.143818,0.438936,0.174948,0.580814,0.181174,0.683998,0.184287,0.426038,0.171835,0.361548,...,0.202966,0.374446,0.262114,0.426038,0.246548,0.567916,0.262114,0.683998,0,1
0,0.143818,0.432487,0.178061,0.580814,0.181174,0.683998,0.187401,0.419589,0.174948,0.361548,...,0.202966,0.374446,0.262114,0.426038,0.246548,0.567916,0.259001,0.683998,0,1
0,0.146931,0.432487,0.178061,0.580814,0.181174,0.683998,0.187401,0.419589,0.174948,0.361548,...,0.202966,0.374446,0.259001,0.426038,0.246548,0.567916,0.262114,0.683998,0,1


In [47]:
y_train=train_data_df["action_gt_num"]
x_train=train_data_df.drop("action_gt_num",axis=1).drop("dummy",axis=1)
x_train.head()

,RShoulder_x_-14,RShoulder_y_-14,RHip_x_-14,RHip_y_-14,RKnee_x_-14,RKnee_y_-14,Neck_x_-14,Neck_y_-14,Nose_x_-14,Nose_y_-14,...,Neck_x_0,Neck_y_0,Nose_x_0,Nose_y_0,LShoulde_x_0,LShoulde_y_0,LHip_x_0,LHip_y_0,LKnee_x_0,LKnee_y_0
0,0.140705,0.432487,0.174948,0.587263,0.181174,0.683998,0.184287,0.426038,0.168722,0.361548,...,0.215418,0.426038,0.196740,0.374446,0.255887,0.426038,0.243435,0.567916,0.259001,0.683998
0,0.143818,0.432487,0.174948,0.587263,0.181174,0.683998,0.184287,0.426038,0.168722,0.361548,...,0.215418,0.426038,0.199853,0.374446,0.259001,0.426038,0.243435,0.567916,0.259001,0.683998
0,0.143818,0.438936,0.174948,0.580814,0.181174,0.683998,0.184287,0.426038,0.171835,0.361548,...,0.215418,0.426038,0.202966,0.374446,0.262114,0.426038,0.246548,0.567916,0.262114,0.683998
0,0.143818,0.432487,0.178061,0.580814,0.181174,0.683998,0.187401,0.419589,0.174948,0.361548,...,0.218531,0.426038,0.202966,0.374446,0.262114,0.426038,0.246548,0.567916,0.259001,0.683998
0,0.146931,0.432487,0.178061,0.580814,0.181174,0.683998,0.187401,0.419589,0.174948,0.361548,...,0.215418,0.426038,0.202966,0.374446,0.259001,0.426038,0.246548,0.567916,0.262114,0.683998


In [48]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(penalty='l2', solver="sag", random_state=0)
clf.fit(x_train, y_train)

C:\Users\81806\anaconda3\envs\pyvino\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)

In [49]:
target_columns = ['RShoulder_x','RShoulder_y','RHip_x','RHip_y','RKnee_x','RKnee_y',
                  'Neck_x','Neck_y','Nose_x','Nose_y','LShoulde_x','LShoulde_y',
                  'LHip_x','LHip_y','LKnee_x','LKnee_y']

In [50]:
frame_length = 15

In [51]:
new_data_df = None
path_file = os.path.join(path_test_file_dir, "test.csv")
data_df = pd.read_csv(path_file,index_col='frame_num')
units_df = None
for idx in data_df.index[frame_length:len(data_df)]:
    unit_df = data_df.loc[idx - frame_length: idx-1]
    y = unit_df[objective]
    Y=y.loc[idx-1]
    X = unit_df.drop(columns=[objective]).loc[:, target_columns]
    flatten_data=X.to_numpy().reshape(1,-1)
    A=['RShoulder_x','RShoulder_y', 'RHip_x', 'RHip_y','RKnee_x',
 'RKnee_y', 'Neck_x', 'Neck_y', 'Nose_x', 'Nose_y', 'LShoulde_x', 'LShoulde_y',
 'LHip_x', 'LHip_y', 'LKnee_x', 'LKnee_y']
    B=[a+"_"+str(i-14) for i in range(15) for a in A]
    X=pd.DataFrame(flatten_data,columns=B)
    X["action_gt_num"]=Y
    unit_df = X
    if units_df is None:
        units_df = unit_df
    else:
        units_df = pd.concat([units_df, unit_df], axis=0, sort=False)
path_save_file = os.path.join(path_save_dir, points_file_name)
units_df.to_csv(path_save_file)
if new_data_df is None:
    new_data_df = units_df
else:
    new_data_df = pd.concat([new_data_df, units_df], axis=0, sort=False)
test_data_df = new_data_df.copy()
test_data_df.to_csv(path_test_file)

In [52]:
test_data_df.groupby(objective).count()

,RShoulder_x_-14,RShoulder_y_-14,RHip_x_-14,RHip_y_-14,RKnee_x_-14,RKnee_y_-14,Neck_x_-14,Neck_y_-14,Nose_x_-14,Nose_y_-14,...,Neck_x_0,Neck_y_0,Nose_x_0,Nose_y_0,LShoulde_x_0,LShoulde_y_0,LHip_x_0,LHip_y_0,LKnee_x_0,LKnee_y_0
action_gt_num,,,,,,,,,,,,,,,,,,,,,
0,337,337,337,337,337,337,337,337,337,337,...,337,337,337,337,337,337,337,337,337,337
1,228,228,228,228,228,228,228,228,228,228,...,228,228,228,228,228,228,228,228,228,228
2,74,74,74,74,74,74,74,74,74,74,...,74,74,74,74,74,74,74,74,74,74
3,117,117,117,117,117,117,117,117,117,117,...,117,117,117,117,117,117,117,117,117,117
4,210,210,210,210,210,210,210,210,210,210,...,210,210,210,210,210,210,210,210,210,210


In [53]:
test_data_df['dummy'] = 1
test_data_df.groupby(objective).count()['dummy']

action_gt_num
0    337
1    228
2     74
3    117
4    210
Name: dummy, dtype: int64

In [55]:
X_test_data_df = test_data_df.drop([objective], axis=1).drop("dummy",axis=1)
y_test_data_df = test_data_df[objective]
X_test_data_df.head()

,RShoulder_x_-14,RShoulder_y_-14,RHip_x_-14,RHip_y_-14,RKnee_x_-14,RKnee_y_-14,Neck_x_-14,Neck_y_-14,Nose_x_-14,Nose_y_-14,...,Neck_x_0,Neck_y_0,Nose_x_0,Nose_y_0,LShoulde_x_0,LShoulde_y_0,LHip_x_0,LHip_y_0,LKnee_x_0,LKnee_y_0
0,0.103776,0.481196,0.110637,0.658869,0.117499,0.779686,0.134652,0.481196,0.083192,0.424341,...,0.117499,0.481196,0.093484,0.410127,0.151805,0.481196,0.148374,0.644655,0.162097,0.765472
0,0.103776,0.481196,0.110637,0.658869,0.117499,0.779686,0.134652,0.481196,0.083192,0.424341,...,0.114068,0.481196,0.090054,0.417234,0.148374,0.481196,0.148374,0.644655,0.162097,0.765472
0,0.107207,0.481196,0.110637,0.658869,0.117499,0.779686,0.134652,0.481196,0.086623,0.424341,...,0.110637,0.481196,0.086623,0.417234,0.144943,0.481196,0.138082,0.651762,0.158666,0.765472
0,0.107207,0.481196,0.110637,0.658869,0.117499,0.779686,0.134652,0.481196,0.086623,0.417234,...,0.107207,0.481196,0.083192,0.417234,0.141513,0.481196,0.134652,0.651762,0.155235,0.765472
0,0.107207,0.481196,0.110637,0.658869,0.117499,0.779686,0.134652,0.481196,0.086623,0.417234,...,0.100345,0.488303,0.079762,0.417234,0.138082,0.481196,0.131221,0.651762,0.148374,0.765472


In [71]:
pred_names=["stand_pred","fall_pred","sittingdown_pred","standingup_pred","sitdown_pred"]
gt_names=["stand_gt","fall_gt","sittingdown_gt","standingup_gt","sitdown_gt"]

In [73]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
predict = clf.predict(X_test_data_df)
print(accuracy_score(y_test_data_df, predict), precision_score(y_test_data_df, predict,average="micro"), recall_score(y_test_data_df, predict,average="micro"))
cm=confusion_matrix(y_test_data_df, predict)
confusion_matrix_df = pd.DataFrame(cm, columns=pred_names, index=gt_names)
confusion_matrix_df

0.6097308488612836 0.6097308488612836 0.6097308488612836


,stand_pred,fall_pred,sittingdown_pred,standingup_pred,sitdown_pred
stand_gt,325,8,0,0,4
fall_gt,0,228,0,0,0
sittingdown_gt,26,12,28,0,8
standingup_gt,40,38,39,0,0
sitdown_gt,10,120,72,0,8


In [74]:
from sklearn.ensemble import GradientBoostingClassifier
model= GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
model.fit(x_train, y_train)
predict= model.predict(X_test_data_df)
print(accuracy_score(y_test_data_df, predict), precision_score(y_test_data_df, predict,average="micro"), recall_score(y_test_data_df, predict,average="micro"))
cm=confusion_matrix(y_test_data_df, predict)
confusion_matrix_df = pd.DataFrame(cm, columns=pred_names, index=gt_names)
confusion_matrix_df

0.5817805383022774 0.5817805383022774 0.5817805383022774


,stand_pred,fall_pred,sittingdown_pred,standingup_pred,sitdown_pred
stand_gt,298,7,3,8,21
fall_gt,0,223,2,0,3
sittingdown_gt,25,8,41,0,0
standingup_gt,49,18,47,0,3
sitdown_gt,6,123,81,0,0


In [75]:
from sklearn.cluster import KMeans
k_means = KMeans(n_clusters=5, random_state=0)
k_means.fit(x_train,y_train)
predict= k_means.predict(X_test_data_df)
print(accuracy_score(y_test_data_df, predict), precision_score(y_test_data_df, predict,average="micro"), recall_score(y_test_data_df, predict,average="micro"))
cm=confusion_matrix(y_test_data_df, predict)
confusion_matrix_df = pd.DataFrame(cm, columns=pred_names, index=gt_names)
confusion_matrix_df

0.3167701863354037 0.3167701863354037 0.3167701863354037


,stand_pred,fall_pred,sittingdown_pred,standingup_pred,sitdown_pred
stand_gt,35,13,258,7,24
fall_gt,14,214,0,0,0
sittingdown_gt,51,0,11,12,0
standingup_gt,63,0,15,39,0
sitdown_gt,80,0,4,119,7


In [76]:
from sklearn.svm import SVC
model = SVC() 
model.fit(x_train, y_train)
predict= model.predict(X_test_data_df)
print(accuracy_score(y_test_data_df, predict), precision_score(y_test_data_df, predict,average="micro"), recall_score(y_test_data_df, predict,average="micro"))
cm=confusion_matrix(y_test_data_df, predict)
confusion_matrix_df = pd.DataFrame(cm, columns=pred_names, index=gt_names)
confusion_matrix_df

0.6086956521739131 0.6086956521739131 0.6086956521739131


,stand_pred,fall_pred,sittingdown_pred,standingup_pred,sitdown_pred
stand_gt,323,8,1,3,2
fall_gt,0,228,0,0,0
sittingdown_gt,27,10,37,0,0
standingup_gt,45,28,44,0,0
sitdown_gt,11,119,80,0,0
